### Autograd

In [13]:
%matplotlib inline
import torch
import numpy as np
import matplotlib.pyplot as plt


In [14]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [15]:
def model(t_u, w, b):
    return w * t_u + b

def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c) ** 2
    return squared_diffs.mean()

params = torch.tensor([1.0, 0.0], requires_grad=True)

In [16]:
params.grad is None

True

Any tensor that will have params as an ancestor will have
access to the chain of functions that were called to get from params to that tensor. In case
these functions are differentiable (and most PyTorch tensor operations will be),the  value
of  the  derivative  will  be  automatically  populated  as  a  grad  attribute  of  the params tensor.

In [17]:
loss = loss_fn(model(t_u, *params), t_c)
loss.backward()
params.grad

tensor([4517.2969,   82.6000])

Let’s repeat together: calling backward will lead derivatives to accumulate at leaf nodes.
So if backward was called earlier, the loss is evaluated again, backward is called again(as  in  any  training  loop),
and  the  gradient  at  each  leaf  is  accumulated  (that  is,summed)  on  top  of  the  one  computed  at  the
previous  iteration,  which  leads  to  an incorrect value for the gradient. In order to prevent this from occurring,
we need to zero the gradient explicitly at each iteration. We can do this easily using the in-place zero_ method

In [18]:
if params.grad is not None:
    params.grad.zero_()

In [20]:
t_un = 0.1 * t_u
def training_loop(n_epochs, learning_rate, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        if params.grad is not None:
            params.grad.zero_()
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)
        loss.backward()

        with torch.no_grad():
            params -= learning_rate * params.grad

        if epoch%500 == 0:
            print('Epoch {}, loss: {}'.format(epoch, float(loss)))
    return params

In [21]:
params = training_loop(
    n_epochs=5000,
    learning_rate=1e-2,
    params=torch.tensor([1.0, 0.0], requires_grad=True),
    t_u = t_un,
    t_c = t_c
)

params

Epoch 500, loss: 7.860115051269531
Epoch 1000, loss: 3.828537940979004
Epoch 1500, loss: 3.092191219329834
Epoch 2000, loss: 2.957697868347168
Epoch 2500, loss: 2.933133840560913
Epoch 3000, loss: 2.9286484718322754
Epoch 3500, loss: 2.9278297424316406
Epoch 4000, loss: 2.9276793003082275
Epoch 4500, loss: 2.927651882171631
Epoch 5000, loss: 2.9276468753814697


tensor([  5.3671, -17.3012], requires_grad=True)